### Importing CybORG

To use the CybORG environment, it is necessary to import the CybORG class. CybORG stands for __Cyb__er __O__perations __R__esearch __G__ym, so remember to capitalise correctly when importing. 

In [1]:
from CybORG import CybORG

c:\Users\c21122256\PycharmProjects\CAGE-Challenge-1\venv\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


### Instantiating CybORG

ALthough CybORG uses an OpenAI gym API, it is not run by calling gym.make(). Instead it has to be manually instantiated. The constructor has two mandatory string parameters: a mode-type which specifies which engine will be used under the hood and the path to a .yaml scenario file which defines the network layout and agent action spaces.

The only currently supported mode is simulation, while this challenge uses Scenario 1b.

In [2]:
import inspect

path = str(inspect.getfile(CybORG))
path = path[:-10] + '/Shared/Scenarios/Scenario1b.yaml'

env = CybORG(path, 'sim')

### Using an Agent with CybORG

Scenario 1b has multiple actors involved: Red team will be attacking the network, Blue team will be defending the network, while Green team represents noise generated by users. Normally the roles of Red and Green will be handled by internal rules-based agents, while Blue team interacts with the external API. However, for demonstration purposes, it will be easier to first examine a Red agent.

CybORG uses an OpenAI Gym interface to interact with agents. Thus, we can begin the scenario by calling the reset method. It is necessary to specify which team you are on as a string parameter. Without using any wrappers, CybORG will return a results object which contains various bits of data. We can get the observation by accessing the corresponding attribute.

In [3]:
results = env.reset(agent='Red')
obs = results.observation
print(obs)

{'success': <TrinaryEnum.UNKNOWN: 2>, 'User0': {'Interface': [{'Interface Name': 'eth0', 'IP Address': IPv4Address('10.0.4.162'), 'Subnet': IPv4Network('10.0.4.160/28')}], 'Sessions': [{'Username': 'SYSTEM', 'ID': 0, 'Timeout': 0, 'PID': 6934, 'Type': <SessionType.RED_ABSTRACT_SESSION: 10>, 'Agent': 'Red'}], 'Processes': [{'PID': 6934, 'Username': 'SYSTEM'}], 'System info': {'Hostname': 'User0', 'OSType': <OperatingSystemType.WINDOWS: 2>, 'OSDistribution': <OperatingSystemDistribution.WINDOWS_SVR_2008: 4>, 'OSVersion': <OperatingSystemVersion.W6_1_7601: 13>, 'Architecture': <Architecture.x64: 2>}}}


We can see that the above observation outputs a messy dictionary. In order to understand raw CybORG observations, please go to the observation tutorial. We will show how to train a neural network-based agent with CybORG down below.

Because of the complexties of Cybersecurity, the action space in CybORG is generated on the fly. For Scenario 1b, this only needs to be extracted at the beginning of the scenario. This can also be found in the results object. It is another messy dictionary, so we will only print out the keys. You can learn more in the action_space tutorial.

In [4]:
action_space = results.action_space
print(action_space)

print(list(action_space.keys()))

{'action': {<class 'CybORG.Shared.Actions.Action.Sleep'>: True, <class 'CybORG.Shared.Actions.AbstractActions.DiscoverRemoteSystems.DiscoverRemoteSystems'>: True, <class 'CybORG.Shared.Actions.AbstractActions.DiscoverNetworkServices.DiscoverNetworkServices'>: True, <class 'CybORG.Shared.Actions.AbstractActions.ExploitRemoteService.ExploitRemoteService'>: True, <class 'CybORG.Shared.Actions.AbstractActions.PrivilegeEscalate.PrivilegeEscalate'>: True, <class 'CybORG.Shared.Actions.AbstractActions.Impact.Impact'>: True}, 'subnet': {IPv4Network('10.0.197.0/28'): False, IPv4Network('10.0.171.64/28'): False, IPv4Network('10.0.4.160/28'): True}, 'ip_address': {IPv4Address('10.0.197.1'): False, IPv4Address('10.0.197.5'): False, IPv4Address('10.0.197.9'): False, IPv4Address('10.0.197.14'): False, IPv4Address('10.0.171.71'): False, IPv4Address('10.0.171.69'): False, IPv4Address('10.0.171.75'): False, IPv4Address('10.0.171.77'): False, IPv4Address('10.0.4.162'): True, IPv4Address('10.0.4.173'): F

Just like OpenAI gym, CybORG uses a step function to input actions and return results. The method itself requires two string parameters: agent is the name of the team that is taking the action and action is the action being performed.

In [5]:
from CybORG.Agents import B_lineAgent,RedMeanderAgent

agent = B_lineAgent()

action = agent.get_action(obs,action_space)

results = env.step(agent='Red',action=action)

DiscoverRemoteSystems 10.0.4.160/28


The results object contains the new observation, reward and done attributes.

In [6]:
print(results.observation)
print(76*'-')
print(results.action)
print(76*'-')
print(results.done)

{'success': <TrinaryEnum.TRUE: 1>, '10.0.4.162': {'Interface': [{'IP Address': IPv4Address('10.0.4.162'), 'Subnet': IPv4Network('10.0.4.160/28')}]}, '10.0.4.173': {'Interface': [{'IP Address': IPv4Address('10.0.4.173'), 'Subnet': IPv4Network('10.0.4.160/28')}]}, '10.0.4.161': {'Interface': [{'IP Address': IPv4Address('10.0.4.161'), 'Subnet': IPv4Network('10.0.4.160/28')}]}, '10.0.4.169': {'Interface': [{'IP Address': IPv4Address('10.0.4.169'), 'Subnet': IPv4Network('10.0.4.160/28')}]}, '10.0.4.164': {'Interface': [{'IP Address': IPv4Address('10.0.4.164'), 'Subnet': IPv4Network('10.0.4.160/28')}]}}
----------------------------------------------------------------------------
DiscoverRemoteSystems 10.0.4.160/28
----------------------------------------------------------------------------
False


### Adding Opponents

In the example above, only Red Team performed any actions. In order to setup CybORG for the full challenge, we want a Blue Agent to be interacting with the external API while Red and Green take their actions automatically in the background.

We can achieve this by specifying an agents dictionary to pass into CybORG when instantiating the class. Now, whenever the step function is called, the agents will take turn to perform their actions. Note that the turn order is Blue, Green then Red.

In [7]:
from CybORG.Agents import B_lineAgent, GreenAgent, BlueMonitorAgent,RedMeanderAgent

agents = {
    'Red': B_lineAgent,
    'Green': GreenAgent
}

env = CybORG(path,'sim',agents=agents)

results = env.reset(agent='Blue')
obs = results.observation
action_space = results.action_space
agent = BlueMonitorAgent()

for step in range(100):
    action = agent.get_action(obs,action_space=action_space)
    results = env.step(agent='Blue',action=action)
    obs = results.observation
    reward = results.reward
    print(reward)

DiscoverRemoteSystems 10.0.179.112/28
-0.1
DiscoverNetworkServices 10.0.179.116
-0.1
ExploitRemoteService 10.0.179.116
-0.2
PrivilegeEscalate User1
-0.2
DiscoverNetworkServices 10.0.105.27
-0.2
ExploitRemoteService 10.0.105.27
-0.2
PrivilegeEscalate Enterprise1
-1.2
DiscoverRemoteSystems 10.0.105.16/28
-1.2
DiscoverNetworkServices 10.0.105.19
-1.2
ExploitRemoteService 10.0.105.19
-1.2
PrivilegeEscalate Enterprise2
-2.2
DiscoverNetworkServices 10.0.243.194
-2.2
-2.2
PrivilegeEscalate Op_Server0
-3.2
Impact Op_Server0
-13.2
-13.2
-13.2
-13.2
-13.2
-13.2
-13.2
-13.2
-13.2
-13.2
-13.2
-13.2
-13.2
-13.2
-13.2
-13.2
-13.2
-13.2
-13.2
-13.2
-13.2
-13.2
-13.2
-13.2
-13.2
-13.2
-13.2
-13.2
-13.2
-13.2
-13.2
-13.2
-13.2
-13.2
-13.2
-13.2
-13.2
-13.2
-13.2
-13.2
-13.2
-13.2
-13.2
-13.2
-13.2
-13.2
-13.2
-13.2
-13.2
-13.2
-13.2
-13.2
-13.2
-13.2
-13.2
-13.2
-13.2
-13.2
-13.2
-13.2
-13.2
-13.2
-13.2
-13.2
-13.2
-13.2
-13.2
-13.2
-13.2
-13.2
-13.2
-13.2
-13.2
-13.2
-13.2
-13.2
-13.2
-13.2
-13.2
-13.

### Using a Neural Network with CybORG

In order to use a Neural network with CybORG, we will need to import a wrapper. The Challenge Wrapper provides all the functionality needed for the Scenario1b challenge and makes the external api much more in line with OpenAI gym.

In [10]:
from CybORG.Agents.Wrappers import ChallengeWrapper
cyborg = CybORG(path,'sim',agents=agents)
env = ChallengeWrapper(env=cyborg,agent_name='Blue')

obs = env.reset()

print(obs)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]


Not only is the observation now a vector, but we can extract the action and observation spaces exactly like a gym environment.

The step function also behaves like that of OpenAI gym. The info parameter contains a dictionary form of the standard results object for debugging purposes.

In [41]:
action = 0

obs, reward, done, info = env.step(action)

print(obs)
print(76*'-')
print(reward)
print(76*'-')
print(done)
print(76*'-')
print(info)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0.
 0. 0. 0. 0.]
----------------------------------------------------------------------------
-13.1
----------------------------------------------------------------------------
False
----------------------------------------------------------------------------
{'observation': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0]), 'next_observation': None, 'done': False, 'reward': -13.1, 'action': <CybORG.Shared.Actions.Action.Sleep object at 0x000001A9C278C850>, 'info': None, 'parameter_mask': None, 'action_space': 54, 'error': None, 'error_msg': None, 'action_name': None, 'selection_masks': None}
